In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.functional as nn

from tqdm import tqdm
from torch_geometric.data import Data

## Notebook for creating gifs, images, maps...

In [6]:
file_graphs = [dict(np.load(f'../data/{file}')) for file in sorted(os.listdir('../data')) if file[-3:]=='npz']
d0 = file_graphs[1]
d0.keys()

dict_keys(['Damage', 'Concentration', 'Thickness', 'Sigma_0', 'Sigma_1', 'M_VT_x', 'M_VT_y', 'M_wind_x', 'M_wind_y', 'M_ocean_x', 'M_ocean_y', 'x', 'y', 't', 'i'])

In [71]:
#This cell plots the spatial distribution of node indexes incrementally. It helped me to understand the indexes

if not os.path.isdir('../figures/indexes'):
    os.mkdir('../figures/indexes')

vel_norm = np.sqrt(d0['M_VT_x']**2 + d0['M_VT_y']**2)
step = 1000
lenght = vel_norm.shape[0]


for i in range(0,lenght,step):
    fig,ax = plt.subplots(1,1,figsize=(10,10))
    ax.scatter(d0['x'][:i],d0['y'][:i], c=vel_norm[:i], s= 3 ,marker='.',linewidths=.7)
    ax.set_title(f'{i}/{lenght} points')
    plt.savefig(f'../figures/indexes/points_{i}.png')
    plt.close(fig)

In [46]:
#Simple "movie" to see that the ice actually moves :)

if not os.path.isdir('../figures/velocities'):
    os.mkdir('../figures/velocities')

for i,file in enumerate(file_graphs):
    # plot sea ice concentration (for each ELEMENT)
    vel_norm = np.sqrt(file['M_VT_x']**2 + file['M_VT_y']**2)

    fig = plt.figure(figsize=(10,10))
    plt.tripcolor(file['x'], file['y'], vel_norm, triangles=file['t'])
    plt.colorbar()
    plt.gca().set_aspect('equal')
    plt.savefig(f'../figures/velocities/snapshot_{i}.png')
    plt.close(fig)